In [1]:
import datetime as dt
import os
import subprocess
from abc import ABC, abstractmethod
from typing import Tuple
import multiprocessing
import numpy as np
from ConfigSpace import (
    Categorical,
    Configuration,
    ConfigurationSpace,
    Float,
    GreaterThanCondition,
    Integer,
)
from smac import AlgorithmConfigurationFacade, Scenario
from smac.runhistory.dataclasses import TrialValue

from copy import deepcopy
from main import *

In [2]:
portfolio = TSP_GlobalPortfolio(size=2, configspace=LKH_CONFIGURATION_SPACE)
training_instances = [
    TSP_Instance("1.tsp", 20887545.00),
    TSP_Instance("2.tsp", 21134211.00),
    TSP_Instance("3.tsp", 21196547.00),
    TSP_Instance("4.tsp", 21428037.00),
    TSP_Instance("5.tsp", 11020488.00),
]

smac_configuration_space = ConfigurationSpace()
for idx, solver in enumerate(portfolio.solvers):
    for k, v in solver.config.config_space.items():
        v = deepcopy(v)
        v.name = f"{idx}__{v.name}"
        smac_configuration_space.add(v)

In [3]:
scenario = Scenario(smac_configuration_space, deterministic=False)

smac = AlgorithmConfigurationFacade(
    scenario,
    lambda seed: None,
    overwrite=True,
)

for _ in range(4):
    info = smac.ask()

    for k, v in info.config.items():
        idx = int(k.split("__")[0])
        key = k.split("__")[1]
        portfolio.solvers[idx].config[key] = v
    cost, time = portfolio.evaluate(training_instances)
    value = TrialValue(cost=cost, time=time)
    print(f"Cost: {cost}, Time: {value.time}")
    smac.tell(info, value)

[INFO][abstract_initial_design.py:95] Reducing the number of initial configurations from 460 to 25 (max_ratio == 0.25).
[INFO][abstract_initial_design.py:147] Using 1 initial design configurations and 0 additional configurations.
[2024-11-03 23:44:57.519065] Evaluating 5 instances
Cost: 1.6740000000000002, Time: 16.82
[INFO][abstract_intensifier.py:516] Added config 9c5d7a as new incumbent because there are no incumbents yet.
[2024-11-03 23:45:04.950013] Evaluating 5 instances
Cost: 1.6599999999999997, Time: 16.68
[2024-11-03 23:45:14.253644] Evaluating 5 instances
Cost: 100.0, Time: 100.0
[2024-11-03 23:46:27.426227] Evaluating 5 instances
Cost: 100.0, Time: 100.0


In [5]:
for k, v in smac.intensifier.get_incumbent().items():
    idx = int(k.split("__")[0])
    key = k.split("__")[1]
    portfolio.solvers[idx].config[key] = v
cost, time = portfolio.evaluate(training_instances)
cost, time

[2024-11-03 23:47:59.041366] Evaluating 5 instances


(1.652, 16.55)

In [13]:
portfolio.solvers[0].solve(training_instances[3])

[<main.TSP_Solver object at 0x000001CFC8EAD750>][16768][2024-11-03 23:48:46.468793] Solving instance 4.tsp
PARAMETER_FILE = config_16768.par
PROBLEM_FILE = 4.tsp
Successes/Runs = 1/1
Cost.min = 21428037, Cost.avg = 21428037.00, Cost.max = 21428037
Gap.min = 0.0000%, Gap.avg = 0.0000%, Gap.max = 0.0000%
Trials.min = 539, Trials.avg = 539.0, Trials.max = 539
Time.min = 2.28 sec., Time.avg = 2.28 sec., Time.max = 2.28 sec.
Time.total = 2.85 sec.
Press any key to continue . . . 


(2.85, 2.85)